In [67]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
data=yf.download("TATASTEEL.NS",start="2018-01-01",end="2020-01-01")
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-01,69.787956,70.064255,68.549400,68.806641,57.710384,19249201
2018-01-02,68.978134,69.668869,68.144485,69.483086,58.277737,25896915
2018-01-03,69.740318,70.264328,69.668869,70.021378,58.729218,27654851
2018-01-04,70.169052,72.736679,69.740318,72.431801,60.750919,84839667
2018-01-05,72.789078,74.018112,72.646172,73.427414,61.585979,74902157
...,...,...,...,...,...,...
2019-12-24,46.220001,46.470001,45.939999,46.299999,40.558655,112355050
2019-12-26,46.299999,47.200001,46.299999,46.790001,40.987900,182556030
2019-12-27,46.924999,47.180000,46.320000,46.950001,41.128052,111443330


In [68]:
# def ema(data,period,column):
#     return data[column].ewm(span=period,adjust=False).mean()
# data["12_ema"]=ema(data,12,"Close")
# data["26_ema"]=ema(data,26,"Close")
# data['macd']=data["12_ema"]-data["26_ema"]
# data["signal_line"]=ema(data,9,"macd")
# data["daily_return"]=(data["Close"]-data["Open"])/data["Open"]


# # rsi code
# def SMA(data,period):
#     return data.rolling(window=period).mean()
# data["returns"]=(data["Close"]-data["Open"])/data["Open"]
# def rsi(data,period,column):

#    avg_gain = data[column].where(data[column] > 0, 0).rolling(window=14).mean()
#    avg_loss = data[column].where(data[column] < 0, 0).rolling(window=14).mean().abs()
#    RS=avg_gain/avg_loss
#    data["RSI"]=100-(100/(1+RS))
# rsi(data,14,"returns")

# data["signal"]=0
# def signal(data):
#     for x in range(data.shape[0]-1):
#         if data["macd"].iloc[x]<data["signal_line"].iloc[x] and data["macd"].iloc[x+1]>=data["signal_line"].iloc[x+1] and data["RSI"].iloc[x+1]<=30:
#             data["signal"].iloc[x+1]=1
#         elif data["macd"].iloc[x]>data["signal_line"].iloc[x] and data["macd"].iloc[x+1]<=data["signal_line"].iloc[x+1] and data["RSI"].iloc[x+1]>=60:
#             data["signal"].iloc[x+1]=-1

# signal(data)


# bollinger band
def bb(data,period,column):
    std=data[column].rolling(window=20).std()
    data["middle"]=data[column].rolling(window=20).mean()
    data["upper"]=data["middle"]+2*std
    data["lower"]=data["middle"]-2*std
bb(data,20,"Close")
data["signal"]=0
def sig(data,column):
    for x in range(data.shape[0]-1):
        if data["Close"][x]<=data['upper'][x] and data["Close"][x+1]>data['upper'][x+1]:
            data["signal"][x+1]=-1
        elif data["Close"][x]>=data["lower"][x]  and data["Close"][x+1]<data["lower"][x+1]:
            data["signal"][x+1]=1

sig(data,"Close")


     

C:\Users\rakes\AppData\Local\Temp\ipykernel_22816\1040941465.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if data["Close"][x]<=data['upper'][x] and data["Close"][x+1]>data['upper'][x+1]:
C:\Users\rakes\AppData\Local\Temp\ipykernel_22816\1040941465.py:45: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif data["Close"][x]>=data["lower"][x]  and data["Close"][x+1]<data["lower"][x+1]:
C:\Users\rakes\AppData\Local\Temp\ipykernel_22816\1040941465.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [69]:
curr=0
capital=10000000
portfolio=[]
entry=0
no=0
data["portfol"]=0
for x in range(len(data)):
    if curr==0 and data["signal"][x]==1:
        curr=1
        entry=data["Close"][x]
        no=capital//data["Close"][x]
        capital=capital-no*entry
        # data["portfol"][x]=no*entry
        portfolio.append(no*entry)
    elif curr==0 and data["signal"][x]==-1:
        curr=-1
        no=capital//data["Close"][x]
        entry=data["Close"][x]

        # data["portfol"][x]=no*data["Close"][x]
        portfolio.append(no*data["Close"][x])
    elif curr==1 and data["Close"][x]<entry*(1-.01):
        curr=0
        
        # data["portfol"][x]=no*entry*(1-.001)
        portfolio.append(no*entry*(1-.01))
        capital=capital+no*entry*(1-.01)
    elif curr==1 and data["Close"][x]>entry*(1+.05) :
        curr=0
        # data["portfol"][x]=no*entry*(1+.01)
        portfolio.append(no*entry*(1+.05))
        capital=capital+no*entry*(1+.05)
    elif curr==1 and data["signal"][x]==-1:
        curr=0
        # data["portfol"][x]=no*(data["Close"][x])
        portfolio.append(no*data["Close"][x])
        capital=capital+no*(data["Close"][x])
    elif curr==-1 and data["Close"][x]>entry*(1+.01):
        curr=0
        # data["portfol"][x]=no*entry*(1+.001)
        portfolio.append(no*entry*(1+.01))
        capital=capital+no*entry-no*entry*(1+.01)
    elif curr==-1 and data["Close"][x]<entry*(1-.05) :
        curr=0
        # data["portfol"][x]=no*entry*(1-.01)
        portfolio.append(no*entry*(1-.05))
        capital=capital+no*entry-no*entry*(1-.05)
    elif curr==-1 and data["signal"][x]==1:
        curr=0
        # data["portfol"][x]=no*data["Close"][x]
        portfolio.append(no*data["Close"][x])
        capital=capital+no*entry-no*data["Close"][x]     
    elif curr==-1 or curr==1:
        portfolio.append(no*data["Close"][x])
if curr==1 :

    capital=capital+data["Close"][len(data)-1]      
    
elif curr==-1:
    capital=capital+no*entry-no*data["Close"][len(data)-1]
capital   

        

C:\Users\rakes\AppData\Local\Temp\ipykernel_22816\4270537773.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if curr==0 and data["signal"][x]==1:
C:\Users\rakes\AppData\Local\Temp\ipykernel_22816\4270537773.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif curr==0 and data["signal"][x]==-1:
C:\Users\rakes\AppData\Local\Temp\ipykernel_22816\4270537773.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  entry=data["Close"

9758120.309640918

In [70]:
# calculation of max drawdown
def max_drawdown(column):
    # max price till date =pmax
    # max drawdown=mdd
    # maximise "d" to calculate the max drawdown  WHERE  [d=(1-(price_curr)/pmax)]
    pmax=0
    mdd=0
    d=0
    for x in range(len(column)):
        if column[x]>pmax:
            pmax=column[x]
        d=1-((column[x])/pmax)
        if d>mdd:
            mdd=d
    return mdd    
100*max_drawdown(portfolio)

14.829439340824512

In [71]:
# trailing stoploss
curr=0
capital=10000000
portfolio=[]
entry=0
no=0
data["portfol"]=0
pmax=0
pmin=100
for x in range(len(data)):
    if(curr==-1 or curr==1):
        if(data["Close"][x]>pmax) :
            pmax=data["Close"][x]
        if(data["Close"][x]<pmin):
            pmin=data["Close"][x]
    if curr==0 and data["signal"][x]==1:
        curr=1
        
        entry=data["Close"][x]
        pmin=entry
        pmax=entry

        no=capital//data["Close"][x]
        capital=capital-no*entry
        # data["portfol"][x]=no*entry
        portfolio.append(no*entry)
    elif curr==0 and data["signal"][x]==-1:
        curr=-1
        no=capital//data["Close"][x]
        entry=data["Close"][x]
        pmin=entry
        pmax=entry

        # data["portfol"][x]=no*data["Close"][x]
        portfolio.append(no*data["Close"][x])
    elif curr==1 and data["Close"][x]<pmax*(1-.01):
        curr=0
        
        # data["portfol"][x]=no*entry*(1-.001)
        portfolio.append(no*pmax*(1-.01))
        capital=capital+no*pmax*(1-.01)
    elif curr==1 and data["Close"][x]>entry*(1+.05) :
        curr=0
        # data["portfol"][x]=no*entry*(1+.01)
        portfolio.append(no*entry*(1+.05))
        capital=capital+no*entry*(1+.05)
    elif curr==1 and data["signal"][x]==-1:
        curr=0
        # data["portfol"][x]=no*(data["Close"][x])
        portfolio.append(no*data["Close"][x])
        capital=capital+no*(data["Close"][x])
    elif curr==-1 and data["Close"][x]>pmin*(1+.01):
        curr=0
        # data["portfol"][x]=no*entry*(1+.001)
        portfolio.append(no*pmin*(1+.01))
        capital=capital+no*entry-no*pmin*(1+.01)
    elif curr==-1 and data["Close"][x]<entry*(1-.05) :
        curr=0
        # data["portfol"][x]=no*entry*(1-.01)
        portfolio.append(no*entry*(1-.05))
        capital=capital+no*entry-no*entry*(1-.05)
    elif curr==-1 and data["signal"][x]==1:
        curr=0
        # data["portfol"][x]=no*data["Close"][x]
        portfolio.append(no*data["Close"][x])
        capital=capital+no*entry-no*data["Close"][x]     
    elif curr==-1 or curr==1:
        portfolio.append(no*data["Close"][x])
if curr==1 :

    capital=capital+data["Close"][len(data)-1]      
    
elif curr==-1:
    capital=capital+no*entry-no*data["Close"][len(data)-1]
capital   

        

C:\Users\rakes\AppData\Local\Temp\ipykernel_22816\610379630.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if curr==0 and data["signal"][x]==1:
C:\Users\rakes\AppData\Local\Temp\ipykernel_22816\610379630.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif curr==0 and data["signal"][x]==-1:
C:\Users\rakes\AppData\Local\Temp\ipykernel_22816\610379630.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  entry=data["Close"][

12219839.223693851

In [72]:
# calculation of max drawdown
def max_drawdown(column):
    # max price till date =pmax
    # max drawdown=mdd
    # maximise "d" to calculate the max drawdown  WHERE  [d=(1-(price_curr)/pmax)]
    pmax=0
    mdd=0
    d=0
    for x in range(len(column)):
        if column[x]>pmax:
            pmax=column[x]
        d=1-((column[x])/pmax)
        if d>mdd:
            mdd=d
    return mdd    
100*max_drawdown(portfolio)

7.319018230692964

In [73]:
data["returns"]=(data["Close"]-data["Open"])/data["Open"]
# sharpe
def sharpe(data,column):
    std=data[column].std()
    
    sharpe=(np.sqrt(252)*data[column].mean()-0)/std
    return sharpe
sharpe_ratio=sharpe(data,"returns")
print("sharpe_ratio=",sharpe_ratio)

sharpe_ratio= -1.3205793135800465


In [74]:
def heikenA(df):
    df['Hclose'] = (df['Open'] + df['High'] + df['Low'] + df['Close']) / 4

    df['Hopen'] = 0
    for i in range(len(df)):
        if i == 0:
            df['Hopen'].iloc[i] = (df['Open'].iloc[i] + df['Close'].iloc[i])/2
        else:
            df['Hopen'].iloc[i] = (df['Hopen'].iloc[i-1] + df['Hclose'].iloc[i-1])/2

    df['Hhigh'] = df[['High', 'Hopen', 'Hclose']].max(axis=1)
    df['Hlow'] = df[['Low', 'Hopen', 'Hclose']].min(axis=1)
heikenA(data)    
new_data=data[["Hopen","Hhigh","Hlow","Hclose"]].copy()

C:\Users\rakes\AppData\Local\Temp\ipykernel_22816\787452404.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Hopen'].iloc[i] = (df['Open'].iloc[i] + df['Close'].iloc[i])/2
C:\Users\rakes\AppData\Local\Temp\ipykernel_22816\787452404.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '69.29729843139648' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df['Hopen'].iloc[i] = (df['Open'].iloc[i] + df['Close'].iloc[i])/2


In [75]:
new_data

,Hopen,Hhigh,Hlow,Hclose
Date,,,,
2018-01-01,69.297298,70.064255,68.549400,69.302063
2018-01-02,69.299681,69.668869,68.144485,69.068644
2018-01-03,69.184162,70.264328,69.184162,69.923723
2018-01-04,69.553943,72.736679,69.553943,71.269463
2018-01-05,70.411703,74.018112,70.411703,73.220194
...,...,...,...,...
2019-12-24,45.419153,46.470001,45.419153,46.232500
2019-12-26,45.825827,47.200001,45.825827,46.647500
2019-12-27,46.236663,47.180000,46.236663,46.843750


In [76]:
def bb(data,period,column):
    std=data[column].rolling(window=20).std()
    data["middle"]=data[column].rolling(window=20).mean()
    data["upper"]=data["middle"]+2*std
    data["lower"]=data["middle"]-2*std
bb(new_data,20,"Hclose")
new_data["Hsignal"]=0
def sig(new_data,column):
    for x in range(new_data.shape[0]-1):
        if new_data["Hclose"][x]<=new_data['upper'][x] and new_data["Hclose"][x+1]>new_data['upper'][x+1]:
            new_data["Hsignal"][x+1]=-1
        elif new_data["Hclose"][x]>=new_data["lower"][x]  and new_data["Hclose"][x+1]<new_data["lower"][x+1]:
            new_data["Hsignal"][x+1]=1

sig(new_data,"Hclose")

C:\Users\rakes\AppData\Local\Temp\ipykernel_22816\2564153149.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if new_data["Hclose"][x]<=new_data['upper'][x] and new_data["Hclose"][x+1]>new_data['upper'][x+1]:
C:\Users\rakes\AppData\Local\Temp\ipykernel_22816\2564153149.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif new_data["Hclose"][x]>=new_data["lower"][x]  and new_data["Hclose"][x+1]<new_data["lower"][x+1]:
C:\Users\rakes\AppData\Local\Temp\ipykernel_22816\2564153149.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentatio

In [77]:

new_data["Hreturns"]=(new_data["Hclose"]-new_data["Hopen"])/new_data["Hopen"]
new_data

,Hopen,Hhigh,Hlow,Hclose,middle,upper,lower,Hsignal,Hreturns
Date,,,,,,,,,
2018-01-01,69.297298,70.064255,68.549400,69.302063,NaN,NaN,NaN,0,0.000069
2018-01-02,69.299681,69.668869,68.144485,69.068644,NaN,NaN,NaN,0,-0.003334
2018-01-03,69.184162,70.264328,69.184162,69.923723,NaN,NaN,NaN,0,0.010690
2018-01-04,69.553943,72.736679,69.553943,71.269463,NaN,NaN,NaN,0,0.024665
2018-01-05,70.411703,74.018112,70.411703,73.220194,NaN,NaN,NaN,0,0.039887
...,...,...,...,...,...,...,...,...,...
2019-12-24,45.419153,46.470001,45.419153,46.232500,42.478562,46.461169,38.495956,0,0.017908
2019-12-26,45.825827,47.200001,45.825827,46.647500,42.694875,47.090086,38.299663,0,0.017930
2019-12-27,46.236663,47.180000,46.236663,46.843750,42.898625,47.669973,38.127277,0,0.013130


In [78]:
curr=0
capital=10000000
Hportfolio=[]
entry=0
no=0

for x in range(len(data)):
    if curr==0 and new_data["Hsignal"][x]==1:
        curr=1
        entry = data["Close"][x]
        no = capital//data["Close"][x]
        capital=capital-no*entry
        # new_data["Hportfol"][x]=no*entry
        Hportfolio.append(no*entry)
    elif curr==0 and new_data["Hsignal"][x]==-1:
        curr=-1
        no = capital//data["Close"][x]
        entry = data["Close"][x]

        # new_data["Hportfol"][x]=no*new_data["Hsignal"][x]
        Hportfolio.append(no*data["Close"][x])
    elif curr==1 and data["Close"][x]<entry*(1-.01):
        curr=0
        
        # new_data["Hportfol"][x]=no*entry*(1-.001)
        Hportfolio.append(no*entry*(1-.01))
        capital=capital+no*entry*(1-.01)
    elif curr==1 and data["Close"][x]>entry*(1+.05) :
        curr=0
        # new_data["Hportfol"][x]=no*entry*(1+.01)
        Hportfolio.append(no*entry*(1+.05))
        capital=capital+no*entry*(1+.05)
    elif curr==1 and new_data["Hsignal"][x]==-1:
        curr=0
        # new_data["Hportfol"][x]=no*(new_data["Hsignal"][x])
        Hportfolio.append(no*data["Close"][x])
        capital=capital+no*(data["Close"][x])
    elif curr==-1 and data["Close"][x]>entry*(1+.01):
        curr=0
        # new_data["Hportfol"][x]=no*entry*(1+.001)
        Hportfolio.append(no*entry*(1+.01))
        capital=capital+no*entry-no*entry*(1+.01)
    elif curr==-1 and data["Close"][x]<entry*(1-.05) :
        curr=0
        # new_data["Hportfol"][x]=no*entry*(1-.01)
        Hportfolio.append(no*entry*(1-.05))
        capital=capital+no*entry-no*entry*(1-.05)
    elif curr==-1 and new_data["Hsignal"][x]==1:
        curr=0
        # new_data["Hportfol"][x]=no*new_data["Hsignal"][x]
        Hportfolio.append(no*data["Close"][x])
        capital=capital+no*entry-no*data["Close"][x]     
    elif curr==-1 or curr==1:
        Hportfolio.append(no*data["Close"][x])
if curr==1 :

    capital=capital+data["Close"][len(new_data)-1]      
    
elif curr==-1:
    capital=capital+no*entry-no*data["Close"][len(data)-1]
capital   

        

C:\Users\rakes\AppData\Local\Temp\ipykernel_22816\354394061.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if curr==0 and new_data["Hsignal"][x]==1:
C:\Users\rakes\AppData\Local\Temp\ipykernel_22816\354394061.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif curr==0 and new_data["Hsignal"][x]==-1:
C:\Users\rakes\AppData\Local\Temp\ipykernel_22816\354394061.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  entry = dat

12579906.372477682

In [79]:
# calculation of max drawdown
def max_drawdown(column):
    # max price till date =pmax
    # max drawdown=mdd
    # maximise "d" to calculate the max drawdown  WHERE  [d=(1-(price_curr)/pmax)]
    pmax=0
    mdd=0
    d=0
    for x in range(len(column)):
        if column[x]>pmax:
            pmax=column[x]
        d=1-((column[x])/pmax)
        if d>mdd:
            mdd=d
    return mdd    
100*max_drawdown(Hportfolio)

10.989386790893818

In [80]:
# heiken Heishi with trailing stoploss
curr=0
capital=10000000
Hportfolio=[]
entry=0
no=0
pmin=100
pmax=0
for x in range(len(data)):
    if(curr==-1 or curr==1):
        if(data["Close"][x]>pmax) :
            pmax=data["Close"][x]
        if(data["Close"][x]<pmin):
            pmin=data["Close"][x]
    if curr==0 and new_data["Hsignal"][x]==1:
        curr=1
        entry = data["Close"][x]
        no = capital//data["Close"][x]
        capital=capital-no*entry
        # new_data["Hportfol"][x]=no*entry
        Hportfolio.append(no*entry)
    elif curr==0 and new_data["Hsignal"][x]==-1:
        curr=-1
        no = capital//data["Close"][x]
        entry = data["Close"][x]

        # new_data["Hportfol"][x]=no*new_data["Hsignal"][x]
        Hportfolio.append(no*data["Close"][x])
    elif curr==1 and data["Close"][x]<pmax*(1-.01):
        curr=0
        
        # new_data["Hportfol"][x]=no*entry*(1-.001)
        Hportfolio.append(no*pmax*(1-.01))
        capital=capital+no*pmax*(1-.01)
    elif curr==1 and data["Close"][x]>entry*(1+.05) :
        curr=0
        # new_data["Hportfol"][x]=no*entry*(1+.01)
        Hportfolio.append(no*entry*(1+.05))
        capital=capital+no*entry*(1+.05)
    elif curr==1 and new_data["Hsignal"][x]==-1:
        curr=0
        # new_data["Hportfol"][x]=no*(new_data["Hsignal"][x])
        Hportfolio.append(no*data["Close"][x])
        capital=capital+no*(data["Close"][x])
    elif curr==-1 and data["Close"][x]>pmin*(1+.01):
        curr=0
        # new_data["Hportfol"][x]=no*entry*(1+.001)
        Hportfolio.append(no*pmin*(1+.01))
        capital=capital+no*entry-no*pmin*(1+.01)
    elif curr==-1 and data["Close"][x]<entry*(1-.05) :
        curr=0
        # new_data["Hportfol"][x]=no*entry*(1-.01)
        Hportfolio.append(no*entry*(1-.05))
        capital=capital+no*entry-no*entry*(1-.05)
    elif curr==-1 and new_data["Hsignal"][x]==1:
        curr=0
        # new_data["Hportfol"][x]=no*new_data["Hsignal"][x]
        Hportfolio.append(no*data["Close"][x])
        capital=capital+no*entry-no*data["Close"][x]     
    elif curr==-1 or curr==1:
        Hportfolio.append(no*data["Close"][x])
if curr==1 :

    capital=capital+data["Close"][len(new_data)-1]      
    
elif curr==-1:
    capital=capital+no*entry-no*data["Close"][len(data)-1]
capital   

        

C:\Users\rakes\AppData\Local\Temp\ipykernel_22816\3978061421.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if curr==0 and new_data["Hsignal"][x]==1:
C:\Users\rakes\AppData\Local\Temp\ipykernel_22816\3978061421.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif curr==0 and new_data["Hsignal"][x]==-1:
C:\Users\rakes\AppData\Local\Temp\ipykernel_22816\3978061421.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  entry =

4956022268.243929

In [81]:
# sharpe
def sharpe(new_data,column):
    std=new_data[column].std()
    sharpe=(252*new_data[column].mean()-0)/(np.sqrt(252)*std)
    return sharpe
sharpe_ratio=sharpe(new_data,"Hreturns")
print("sharpe_ratio=",sharpe_ratio)

sharpe_ratio= -1.0472719588454575


In [82]:
# calculation of max drawdown
def max_drawdown(column):
    # max price till date =pmax
    # max drawdown=mdd
    # maximise "d" to calculate the max drawdown  WHERE  [d=(1-(price_curr)/pmax)]
    pmax=0
    mdd=0
    d=0
    for x in range(len(column)):
        if column[x]>pmax:
            pmax=column[x]
        d=1-((column[x])/pmax)
        if d>mdd:
            mdd=d
    return mdd    
100*max_drawdown(Hportfolio)

26.287523672224655

In [83]:
# sortino is similar to but std is of negative returns
def sortino(data,column):
    sortino_ratio=np.sqrt(252)*((data[column].mean()) -0)/((data[column][data[column] < 0].std()))
    return sortino_ratio
print(sortino(data,"returns"))
print(sortino(new_data,"Hreturns"))

-2.0570010526923053
-1.6301666356331268


In [84]:
tickers=["TATASTEEL.NS","^NSEI"]
data=pd.DataFrame()
for t in tickers:
    data[t]=yf.download(t,start="2018-01-01",end="2020-01-01")["Close"]
returns=np.log(data/data.shift(1))
market_var=returns["^NSEI"].var()
cov=returns.cov()
cov
beta=cov.iloc[1,0]/market_var
beta

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


1.485524738679312

In [85]:
market_initial_price=4546.20  #of Nifty 50 Index
market_final_price=22530.70
years=16
cagr=((market_final_price/market_initial_price)**(1/16)-1)*100
cagr

10.521147199334347

In [86]:
tickers=["TATASTEEL.NS","^NSEI"]
data=pd.DataFrame()
for t in tickers:
    data[t]=yf.download(t,start="2018-01-01",end="2020-01-01")["Close"]
data["TATASTEEL.NS"]=new_data["Hclose"]
returns=np.log(data/data.shift(1))
market_var=returns["^NSEI"].var()
cov=returns.cov()
cov
beta2=cov.iloc[1,0]/market_var
beta2

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


0.954358156439643

In [87]:
data["trade_returns"]=0

alpha=4.3-beta*cagr
alpha

-11.329424443897732

In [88]:
new_data["trade_returns"]=0

alpha2= 7.34-beta2*cagr
alpha2

-2.7009426447868403